# Group Structure and Order

**Module 06** | 06-elliptic-curves

*E.order(), Hasse bound, abelian_group()*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **group structure and order**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Curves over Finite Fields](06c-curves-over-finite-fields.ipynb).
- Concepts and notation introduced in the previous notebook.

## Counting Points: Hasse's Theorem

In [ ]:
# |E(GF(p))| is close to p+1: |#E - (p+1)| <= 2*sqrt(p)
p = 101
E = EllipticCurve(GF(p), [1, 1])
n = E.order()
print(f'#E = {n}, p+1 = {p+1}, bound = {2*isqrt(p)}')

## Group Structure

In [ ]:
# The group E(GF(p)) is isomorphic to Z/n1 x Z/n2
p = 101
E = EllipticCurve(GF(p), [1, 1])
print(E.abelian_group())

## Point Orders

In [ ]:
# Find the order of individual points
p = 101
E = EllipticCurve(GF(p), [1, 1])
P = E.random_point()
print(f'Order of P: {P.order()}')

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **group structure and order**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Scalar Multiplication](06e-scalar-multiplication.ipynb)